In [2]:
import pandas as pd
import plotly.express as px
import datasets

In [50]:
# load zero-shot direct prompt datasets (baseline)

ZERO_SHOT_EXPERIMENTS = [
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_zero_shot_direct_1699902139",
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_zero_shot_direct_1699889432",
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_zero_shot_direct_1699844874",
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_zero_shot_direct_1699828513",
]

ZERO_SHOT_DATASETS = []

for i, path in enumerate(ZERO_SHOT_EXPERIMENTS):
    ds = datasets.load_from_disk(path).with_format("pandas")
    df = pd.DataFrame(ds["zero_shot_direct__accuracy"])
    df["experiment_num"] = i
    ZERO_SHOT_DATASETS.append(df)


ZERO_SHOT_DF = (
    pd.concat(ZERO_SHOT_DATASETS)
    .groupby(["experiment_num", "zero_shot_direct__accuracy"])
    .value_counts()
    .reset_index()
)
# ZERO_SHOT_DATASETS
print(ZERO_SHOT_DF)
fig = px.bar(ZERO_SHOT_DF, x="experiment_num", y="count", color="zero_shot_direct__accuracy")
fig.show()

   experiment_num  zero_shot_direct__accuracy  count
0               0                       False    970
1               0                        True    349
2               1                       False    995
3               1                        True    324
4               2                       False    979
5               2                        True    340
6               3                       False    980
7               3                        True    339


In [38]:
ds = datasets.load_from_disk(ZERO_SHOT_EXPERIMENTS[0]).with_format("pandas")
df = pd.DataFrame(ds["zero_shot_direct__accuracy"])
df["experiment_num"] = 0
df

,zero_shot_direct__accuracy,experiment_num
0,True,0
1,False,0
2,False,0
3,True,0
4,False,0
...,...,...
1314,True,0
1315,True,0
1316,False,0
1317,False,0


In [49]:
ZERO_SHOT_DF

experiment_num  zero_shot_direct__accuracy
0               False                         970
                True                          349
1               False                         995
                True                          324
2               False                         979
                True                          340
3               False                         980
                True                          339
Name: count, dtype: int64

In [ ]:
# load zero-shot cot

ZERO_SHOT_COT_EXPERIMENTS = [
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_few_shot_auto_cot_num_examples=0_1699981575",
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_few_shot_auto_cot_num_examples=0_1699940749",
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_few_shot_auto_cot_num_examples=0_1699938071",
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_few_shot_auto_cot_num_examples=0_1699986571",
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_few_shot_auto_cot_num_examples=0_1699991315",
]

ONE_SHOT_COT_EXPERIMENTS = [
    "/mnt/spindle/stanford-ssg-research/.cache/experiments/gsm8k_meta-llama/Llama-2-7b-chat-hf_few_shot_auto_cot_num_examples=1_1700007401",
]

In [ ]:
# load n-shot cot

In [ ]:
# load 2x n-shot cot